In [1]:
import pandas as pd
import numpy as np
import os
from __future__ import annotations
from dataclasses import dataclass, field
from pathlib import Path
from typing import Optional
from collections import OrderedDict
import gc
from tqdm import tqdm

import torch
import torch.nn as nn
import timm
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [2]:
@dataclass
class InferenceConfig:
    test_csv  = Path("/kaggle/input/csiro-biomass/test.csv")
    test_img = Path("/kaggle/input/csiro-biomass/test")
    model_dir = Path("/kaggle/input/5fold-pytorch-model-weights")
    model_name = "vit_huge_plus_patch16_dinov3.lvd1689m"
    img_size : int = 518
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    GPU = torch.cuda.get_device_name(0)
    folds = 5
    train_target_cols: list[str] = field(default_factory=lambda: [
        'Dry_Total_g', 'GDM_g', 'Dry_Green_g'
    ])
    all_target_cols: list[str] = field(default_factory=lambda: [
        'Dry_Green_g', 'Dry_Dead_g', 'Dry_Clover_g', 'GDM_g', 'Dry_Total_g'
    ])

    def display_info(self):
        print(f"{'='*70}")
        print(f"Inference Configuration")
        print(f"{'='*70}")
        print(f"Device: {self.device}")
        print(f"GPU: {self.GPU}")
        print(f"Backbone: {self.model_name}")
        print(f"Image Size:{self.img_size}")
        print(f"Folds:{self.folds}")
        print("TTA : 4 views")
        print(f"{'='*70}\n")

class TTAFactory():
    def __init__(self,img_size:int):
        self.img_size = img_size
        self.normal_transform = [
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ]
    def get_TTA(self) -> list[A.Compose]:

        
        original = A.Compose([
            A.Resize(self.img_size,self.img_size),
            *self.normal_transform
        ])

        hflip = A.Compose([
            A.HorizontalFlip(p=1.0),
            A.Resize(self.img_size,self.img_size),
            *self.normal_transform
        ])


        vflip = A.Compose([
            
            A.VerticalFlip(p=1.0),
            A.Resize(self.img_size,self.img_size),
            *self.normal_transform
        ])

        rotate = A.Compose([
            
            A.RandomRotate90(p=1.0),
            A.Resize(self.img_size,self.img_size),
            *self.normal_transform
         ])

        return [original,hflip,vflip,rotate]

class TestDataset(Dataset):
    def __init__(self,df:pd.Dataframe,transform_on:A.Compose,img_dir:Path):
        self.df = df
        self.transform = transform_on
        self.img_dir = img_dir
        self.img_paths = df['image_path'].values

    def __len__(self) -> int:
        return len(self.df)

    def __getitem__(self,idx:int) -> tuple[torch.Tensor,torch.Tensor]:
        img_pth = self.img_paths[idx]
        full_pth = self.img_dir/ Path(img_pth).name

        image = cv2.imread(str(full_pth))
        if image is None:
            print(f"Warning: Failed to load image: {full_pth}")

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        height, width = image.shape[:2]
        mid_point = width // 2
        img_left = image[:, :mid_point]
        img_right = image[:, mid_point:]

        img_left_tensor = self.transform(image=img_left)['image']
        img_right_tensor = self.transform(image=img_right)['image']
        
        return img_left_tensor, img_right_tensor

class Regressor(nn.Module):
    def __init__(self,model_name,pretrained=True):
        super().__init__()
        self.backbone = timm.create_model(model_name,pretrained=pretrained,num_classes=0)
        self.n_features = self.backbone.num_features
        self.n_combined = self.n_features * 2

        def make_head():
            return nn.Sequential(
                nn.Linear(self.n_combined,self.n_combined // 2),
                nn.GELU(),
                nn.Dropout(0.1),
                nn.Linear(self.n_combined // 2,1)
            )

        self.head_total = make_head()
        self.head_gdm = make_head()
        self.head_green = make_head()

    def forward(self,img_left:torch.tensor,img_right:torch.tensor):
        
        feat_left = self.backbone(img_left)
        feat_right = self.backbone(img_right)

        combined = torch.cat([feat_left,feat_right], dim=1)
        out_total = self.head_total(combined)
        out_gdm = self.head_gdm(combined)
        out_green = self.head_green(combined)

        return out_total,out_gdm,out_green

class ModelLoader:
    def __init__(self,config:InferenceConfig):
        self.config = config

    def load_fold_models(self) -> list[nn.Module]:
        print(f"\nLoading {self.config.folds} trained models...")

        models = []

        for fold in range(self.config.folds):
            model_pth = self.config.model_dir / f'best_model_fold{fold}.pth'
            
            if not model_pth.exists():
                raise FileNotFoundError(f"Model file not found: {model_pth}")

            base_model = Regressor(self.config.model_name,pretrained=False)
            state_dict  = torch.load(model_pth, map_location=self.config.device)
            base_model.load_state_dict(state_dict)
            base_model.eval()  
            base_model.to(self.config.device)

            models.append(base_model)
            print(f"  ✓ Fold {fold} model loaded")


        print(f"✓ Successfully loaded {len(models)} models\n")
        return models

class InferenceEngine:
    def __init__(self,models: list[nn.Module],config:InferenceConfig):
        self.models = models
        self.config = config


    def predict_single_view(self,loader:DataLoader) -> dict[str,np.ndarray]:
        view_preds = {'total': [], 'gdm': [], 'green': []}
        with torch.no_grad():
            for img_left, img_right in tqdm(loader, desc="  Predicting", leave=False):
                img_left = img_left.to(self.config.device)
                img_right = img_right.to(self.config.device)

                fold_preds = {'total': [], 'gdm': [], 'green': []}
                
                for model in self.models:
                    pred_total, pred_gdm, pred_green = model(img_left, img_right)
                    fold_preds['total'].append(pred_total.cpu())
                    fold_preds['gdm'].append(pred_gdm.cpu())
                    fold_preds['green'].append(pred_green.cpu())
                    
                avg_total = torch.mean(torch.stack(fold_preds['total']), dim=0)
                avg_gdm = torch.mean(torch.stack(fold_preds['gdm']), dim=0)
                avg_green = torch.mean(torch.stack(fold_preds['green']), dim=0)

                view_preds['total'].append(avg_total.numpy())
                view_preds['gdm'].append(avg_gdm.numpy())
                view_preds['green'].append(avg_green.numpy())

        return {
            k: np.concatenate(v).flatten()
            for k, v in view_preds.items()
        }

    def predict_with_tta(self,test_df:pd.DataFrame,tta_transforms:list[A.Compose]) -> dict[str,np.ndarray]:
        
         
        print(f"\nStarting TTA inference: {len(tta_transforms)} Views × {self.config.folds} Folds")
        all_view_preds: list[dict[str, np.ndarray]] = []
        for i, transform in enumerate(tta_transforms):
            
            print(f"--- TTA View {i+1}/{len(tta_transforms)} ---")

            dataset = TestDataset(test_df,transform,self.config.test_img)
            loader = DataLoader(dataset,batch_size=16,shuffle=False,num_workers = 1)

            view_preds = self.predict_single_view(loader)
            all_view_preds.append(view_preds)

            print(f"  ✓ View {i+1} completed")

        print("\nCalculating TTA Ensemble (averaging all views)...")
        final_preds = {
            
            'total': np.mean([p['total'] for p in all_view_preds], axis=0),
            'gdm': np.mean([p['gdm'] for p in all_view_preds], axis=0),
            'green': np.mean([p['green'] for p in all_view_preds], axis=0)
        }
        
        print("✓ Inference completed\n")
        return final_preds

class submissioncalculator:
    def __init__(self,config:InferenceConfig):
        self.config = config

    def create(self,predictions:dict[str, np.ndarray],test_df_long: pd.DataFrame,test_df_unique: pd.DataFrame) -> None:
        print("Creating Submission CSV")
        pred_total = predictions["total"]
        pred_gdm = predictions["gdm"]
        pred_green = predictions["green"]

        pred_clover = np.maximum(0, pred_gdm - pred_green)
        pred_dead = np.maximum(0, pred_total - pred_gdm)

        preds_wide = pd.DataFrame({
            'image_path': test_df_unique['image_path'],
            'Dry_Green_g': pred_green,
            'Dry_Dead_g': pred_dead,
            'Dry_Clover_g': pred_clover,
            'GDM_g': pred_gdm,
            'Dry_Total_g': pred_total
        })

        preds_long = preds_wide.melt(
            id_vars=['image_path'],
            value_vars=self.config.all_target_cols,
            var_name='target_name',
            value_name='target'
        )
        submission = pd.merge(
            test_df_long[['sample_id', 'image_path', 'target_name']],
            preds_long,
            on=['image_path', 'target_name'],
            how='left'
        )
        submission = submission[['sample_id', 'target']]
        submission.to_csv("submission.csv", index=False)
        print("\n🎉 Submission Saved")
        print("\n--- First 5 rows ---")
        print(submission.head())


class PredictionPipeline:
    def __init__(self,config:InferenceConfig):
        self.config = config
        self.model_loader = ModelLoader(config)
        self.tta_factory = TTAFactory(config.img_size)
        self.sub_creator = submissioncalculator(config)

    def run(self) -> None:
        print(f"\n{'='*70}")
        print(f"🚀 Starting Inference Pipeline")
        print(f"{'='*70}")

        try:
            test_df_long, test_df_unique = self.load_test_data()
            models = self.model_loader.load_fold_models()
            engine = InferenceEngine(models, self.config)
            tta_transforms = self.tta_factory.get_TTA()
            predictions = engine.predict_with_tta(test_df_unique, tta_transforms)

            self.sub_creator.create(
                predictions,
                test_df_long,
                test_df_unique
            )

            print("\n✨ Inference Pipeline Completed ✨")

        except Exception as e:
            print(f"\n❌ Error occurred: {e}")
            raise

        finally:
            
            gc.collect()
            torch.cuda.empty_cache()

    def load_test_data(self) -> tuple[pd.DataFrame,pd.DataFrame]:
        print(f"\nLoading test data: {self.config.test_csv}")
        if not self.config.test_csv.exists():
            raise FileNotFoundError(f"test.csv not found: {self.config.test_csv}")
                                    
            
                                    
        test_df_long = pd.read_csv(self.config.test_csv)
        test_df_unique = test_df_long.drop_duplicates(
            subset=['image_path']
            ).reset_index(drop=True)
        print(f"  Long format: {len(test_df_long)} rows")
        print(f"  Unique images: {len(test_df_unique)} images\n")
        
        return test_df_long, test_df_unique

if __name__ == '__main__':
    config = InferenceConfig()
    config.display_info()

    pipeline = PredictionPipeline(config)
    pipeline.run()
    print("="*70)

Inference Configuration
Device: cuda
GPU: Tesla P100-PCIE-16GB
Backbone: vit_large_patch14_dinov2.lvd142m
Image Size:518
Folds:5
TTA : 4 views


🚀 Starting Inference Pipeline

Loading test data: /kaggle/input/csiro-biomass/test.csv
  Long format: 5 rows
  Unique images: 1 images


Loading 5 trained models...
  ✓ Fold 0 model loaded
  ✓ Fold 1 model loaded
  ✓ Fold 2 model loaded
  ✓ Fold 3 model loaded
  ✓ Fold 4 model loaded
✓ Successfully loaded 5 models


Starting TTA inference: 4 Views × 5 Folds
--- TTA View 1/4 ---


  ✓ View 1 completed
--- TTA View 2/4 ---


  ✓ View 2 completed
--- TTA View 3/4 ---


  ✓ View 3 completed
--- TTA View 4/4 ---


  ✓ View 4 completed

Calculating TTA Ensemble (averaging all views)...
✓ Inference completed

Creating Submission CSV

🎉 Submission Saved

--- First 5 rows ---
                    sample_id     target
0  ID1001187975__Dry_Clover_g   4.094429
1    ID1001187975__Dry_Dead_g  26.391701
2   ID1001187975__Dry_Green_g  30.003250
3   ID1001187975__Dry_Total_g  60.489380
4         ID1001187975__GDM_g  34.097679

✨ Inference Pipeline Completed ✨
